In [4]:
#importer les bibliothèques

In [37]:
import warnings
UserWarning("ignore")

UserWarning('ignore')

In [38]:
import json
import pandas as pd
import datetime
import tabula.io
from tabula.io import read_pdf
import wget

from tabulate import tabulate
import os
import copy

In [6]:
#télécharger les fichiers JSON des villes & regions

In [7]:
wget.download("https://raw.githubusercontent.com/alaouy/sql-moroccan-cities/master/json/ville.json")

'ville (1).json'

In [8]:
wget.download("https://raw.githubusercontent.com/alaouy/sql-moroccan-cities/master/json/region.json")

'region (1).json'

In [9]:
#construite un pandas data frame pour les villes et les régions

In [10]:
cities = pd.read_json("ville.json",orient="records")
cities.rename(columns={'id': 'id_ville', 'region': 'id_region'}, inplace=True)

In [11]:
regions = pd.read_json("region.json",orient="records")
regions.rename(columns={'id': 'id_region'}, inplace=True)

In [12]:
cities_regions = pd.merge(cities,regions)
cities_regions.head(2)

,id_ville,ville,id_region,region
0,0,Afourar,5,Béni Mellal-Khénifra
1,3,Aghbala,5,Béni Mellal-Khénifra


In [13]:
#sur Linux WSL / google collab test du téléchargement pour un seul fichier

In [14]:
wget.download("http://81.192.10.228/wp-content/uploads/2022/03/1_3_2022.pdf",out="./in")

'./in/1_3_2022.pdf'

In [15]:
#parametrage du téléchargement pour plusieurs fichiers sur une marge de date

In [16]:
start = datetime.datetime(2021,3,13)
end = datetime.datetime(2021,3,20)

In [17]:
#téléchargement de tout les fichiers vers un dossier in/

In [18]:
d = start
c=1
while(d<end):
    try:
        print(d.year,d.month,d.day)
        month = str(d.month)
        if d.month<10:
            month = "0"+month
        wget.download(f"http://81.192.10.228/wp-content/uploads/{d.year}/{month}/{d.day}_{d.month}_{d.year}.pdf", out ="./in")
        print(f"http://81.192.10.228/wp-content/uploads/{d.year}/{month}/{d.day}_{d.month}_{d.year}.pdf")
    except Exception as e:
        print("\n-> ERROR!!!!",e)
        
    d = d+datetime.timedelta(days=1)
  
    print("-----------")

2021 3 13
http://81.192.10.228/wp-content/uploads/2021/03/13_3_2021.pdf
-----------
2021 3 14
http://81.192.10.228/wp-content/uploads/2021/03/14_3_2021.pdf
-----------
2021 3 15
http://81.192.10.228/wp-content/uploads/2021/03/15_3_2021.pdf
-----------
2021 3 16
http://81.192.10.228/wp-content/uploads/2021/03/16_3_2021.pdf
-----------
2021 3 17
http://81.192.10.228/wp-content/uploads/2021/03/17_3_2021.pdf
-----------
2021 3 18
http://81.192.10.228/wp-content/uploads/2021/03/18_3_2021.pdf
-----------
2021 3 19
http://81.192.10.228/wp-content/uploads/2021/03/19_3_2021.pdf
-----------


In [19]:
#lister tout les fichiers téléchargés

In [20]:
import os
import re
filepath="./in/" #root path for checking
full_path = filepath
filepattern=r"\b(\w*.pdf)"
filepattern = full_path.replace("/","\/")+filepattern
file_pattern = re.compile(filepattern)

print("Searching...")
counter=0
tables=[]
try:
  #######################################
    for dirpath, subdirs, files in os.walk(full_path):
        for x in files:
            if x.endswith(".pdf"):        
                if re.match(file_pattern, os.path.join(dirpath, x)):
                    data = os.path.join(dirpath, x).split("/")[-1].split(".")[0]
                    tables.append(data.upper())
                    counter+=1
#                     print("")
#                     context['task_instance'].xcom_push('file_name', files)
#                     return True
    print(f"ALL DONE! {counter} sources/tables found")
#     return False
except:
    print("OOOOPS ERORRS ARE HERE!")
#     return False
# print("\n\n",tables)

Searching...
ALL DONE! 8 sources/tables found


In [21]:
# Conversion de tout les fichiers téléchargés vers des csv non nettoyés

In [22]:

for file in tables:
    try:
        print(file+".pdf")        
        tabula.io.convert_into(f"./in/{file}.pdf",f"./out/{file}.csv",output_format="csv",pages="all")

    except Exception as e:
        print("\n-> ERROR!!!!",e) 
    print("-----------")


13_3_2021.pdf
-----------
14_3_2021.pdf
-----------
15_3_2021.pdf
-----------
16_3_2021.pdf
-----------
17_3_2021.pdf
-----------
18_3_2021.pdf
-----------
19_3_2021.pdf
-----------
1_3_2022.pdf
-----------


In [23]:
#dev code abandonné

In [24]:

#tabula.convert_into("1_3_2022.pdf","output.csv",output_format="csv",pages="all")

# import camelot
 
# # extract all the tables in the PDF file
# abc = camelot.read_pdf("1_3_2022.pdf")   #address of file location
 
# # print the first table as Pandas DataFrame
# print(abc[0].df)

In [25]:
# from csv import reader
# # open file in read mode
# with open('./out/2_11_2021.csv', 'r') as read_obj:
#     # pass the file object to reader() to get the reader object
#     csv_reader = reader(read_obj)
#     # Iterate over each row in the csv using reader object
#     all_rows = []
#     for row in csv_reader:
#         # row variable is a list that represents a row in csv
#         all_rows.append(row)
# data = all_rows[4:]
# columns = ["BARRAGES","CAPACITE NORMALE (Mm3)","RESERVE (Mm3)","TAUX DE REMPLISSAGE ( % )","RESERVE (Mm3) Last Year","TAUX DE REMPLISSAGE ( % ) Last Year"]
# df = pd.DataFrame(data=data,columns=columns)
# df["date"] =  datetime.datetime.strptime("2_11_2021", '%d_%m_%Y')
# df

In [26]:
#lecture de tout les csv en sorti avec la lib csv et la concaténation de toute la data dans un seul csv

In [27]:
dfs = []
for file in tables:
    from csv import reader
    # open file in read mode
    with open(f'./out/{file}.csv', 'r') as read_obj:
        # pass the file object to reader() to get the reader object
        csv_reader = reader(read_obj)
        # Iterate over each row in the csv using reader object
        all_rows = []
        for row in csv_reader:
            # row variable is a list that represents a row in csv
            all_rows.append(row)
    data = all_rows[4:]
    columns = ["barrages","capacite_normal_mm3","reserve_mm3","taux_de_remplissage_percent","reserve_mm3_last_year","taux_de_remplissage_percent_last_year"]
    df = pd.DataFrame(data=data,columns=columns)
    df["date"] =  datetime.datetime.strptime(file, '%d_%m_%Y').strftime("%Y-%m-%d")
    dfs.append(copy.copy(df))

In [28]:
count=0
for i in range(len(dfs)):
    count+=len(dfs[i])
print(count)

500


In [29]:
all_data = pd.concat(dfs)
all_data = all_data.reset_index()
all_data = all_data.drop(["index"],axis=1)
all_data

,barrages,capacite_normal_mm3,reserve_mm3,taux_de_remplissage_percent,reserve_mm3_last_year,taux_de_remplissage_percent_last_year,date
0,OUED EL MAKHAZINE,"672,9","670,0","99,6","652,6","97,0",2021-03-13
1,TANGER - MEDITERRANEE,"22,0","19,1","86,8","13,2","60,2",2021-03-13
2,IBN BATTOUTA,"29,1","29,1","100,0","5,4","18,7",2021-03-13
3,NEUF AVRIL 1947,"300,0","76,5","25,5","75,2","25,1",2021-03-13
4,NAKHLA,"4,21","4,11","97,6","4,2","99,1",2021-03-13
...,...,...,...,...,...,...,...
495,SULTAN MOULAY ALI CHERIF,"280,2","93,2","33,3","160,7","57,3",2022-03-01
496,HASSAN ADDAKHIL,"312,8","52,4","16,8","53,1","17,0",2022-03-01
497,SAQUIAT ALHAMRA,-,-,-,-,-,2022-03-01
498,TWUIZGUI RAMZ,"15,0","0,0","0,0","0,0","0,0",2022-03-01


In [30]:
all_data["barrages"].unique()

array(['OUED EL MAKHAZINE', 'TANGER - MEDITERRANEE', 'IBN BATTOUTA',
       'NEUF AVRIL 1947', 'NAKHLA', 'MY HASSAN BEN AL MAHDI', 'SMIR',
       'M.B.A. AL KHATTABI', 'CHEFCHAOUEN', 'ACHARIF AL IDRISSI',
       'HASSAN II', 'Dar Khrofa', 'JOUMOUA', 'INJIL',
       'BARRAGE SUR OUED ZA', 'MOHAMED V', 'MECHRAA HOMMADI', 'BAB LOUTA',
       'IDRISS 1 er', 'ALLAL EL FASSI', 'SIDI ECHAHED', 'SAHLA',
       'BOUHOUDA', 'ASFALOU', 'ALWAHDA', 'GARDE SEBOU', 'EL KENSERA',
       'MICHLIFEN', 'AL HIMER', 'S.M.  BEN ABDEELLAH', 'TAMESNA',
       'OUED MELLAH', 'BIN EL OUIDANE', 'HASSAN 1er', 'SIDI DRISS',
       'MOULAY YOUSSEF', 'TIMINOUTINE', 'AHMED AL HANSSALI',
       'AIT MESSAOUD', 'AL MASSIRA', 'IMFOUT', 'DAOURAT',
       'SIDI SAID MAACHOU', "S.M'HAMED BEN SLIMAN JAZOULI",
       'YAAKOUB MANSOUR', 'ABOU EL ABBAS ESSABTI', 'LALLA TAKERKOUST',
       'BARRAGE MOULAY ABDERRAHMANE', 'DKHILA', 'MOULAY ABDALLAH',
       'AOULOUZ', 'ABDELMOUMENE', 'MOKHTAR SOUSSI', 'IMI EL KHENG',
       'AHL 

In [31]:
#choisir vos barrages (le plus proches de votre ville natale )

In [32]:
selected_barages = ["LALLA TAKERKOUST"]

In [33]:
selected_data = all_data[all_data["barrages"].isin(selected_barages)]
selected_data.reset_index(drop=True, inplace=True)
selected_data["ville"] = ""
selected_data["region"] = ""
selected_data.loc[selected_data['barrages'] == 'LALLA TAKERKOUST', 'ville'] = "Marrakech"
selected_data.loc[selected_data['barrages'] == 'LALLA TAKERKOUST', 'region'] = "Marrakech-Safi "

C:\Users\hp\AppData\Local\Temp/ipykernel_904/456276291.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data["ville"] = ""
C:\Users\hp\AppData\Local\Temp/ipykernel_904/456276291.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data["region"] = ""


In [34]:
selected_data

,barrages,capacite_normal_mm3,reserve_mm3,taux_de_remplissage_percent,reserve_mm3_last_year,taux_de_remplissage_percent_last_year,date,ville,region
0,LALLA TAKERKOUST,"53,3","16,6","31,1","3,6","6,7",2021-03-13,Marrakech,Marrakech-Safi
1,LALLA TAKERKOUST,"53,3","16,6","31,1","3,6","6,7",2021-03-14,Marrakech,Marrakech-Safi
2,LALLA TAKERKOUST,"53,3","16,6","31,1","3,4","6,5",2021-03-15,Marrakech,Marrakech-Safi
3,LALLA TAKERKOUST,"53,3","16,6","31,1","3,4","6,5",2021-03-16,Marrakech,Marrakech-Safi
4,LALLA TAKERKOUST,"53,3","16,6","31,1","3,4","6,5",2021-03-17,Marrakech,Marrakech-Safi
5,LALLA TAKERKOUST,"53,3","16,6","31,1","3,4","6,5",2021-03-18,Marrakech,Marrakech-Safi
6,LALLA TAKERKOUST,"53,3","16,6","31,1","3,4","6,5",2021-03-19,Marrakech,Marrakech-Safi
7,LALLA TAKERKOUST,"53,3","3,2","6,1","7,2","13,5",2022-03-01,Marrakech,Marrakech-Safi


In [35]:
#souvegarde de votre data

In [36]:
selected_data.to_csv("./barrage_data.csv",index=False,encoding="UTF-8")